<a href="https://colab.research.google.com/github/srilamaiti/fall_2023_210_capstone_deepika_srila/blob/main/final_deliverable/model/inferencing/video_inference_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install moviepy
!pip install ffmpy

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5579 sha256=5fcfdc3306df91f9804b4d57b9099eeaa3f8a369b7312eea85e0a6d69b04d6d9
  Stored in directory: /root/.cache/pip/wheels/01/a6/d1/1c0828c304a4283b2c1639a09ad86f83d7c487ef34c6b4a1bf
Successfully built ffmpy


In [2]:
import os
from google.colab.patches import cv2_imshow
import ffmpy
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display
from urllib import request
from IPython.display import HTML
from base64 import b64encode
import moviepy.video.io.ImageSequenceClip
import cv2

import tensorflow as tf
from collections import deque
import time

In [3]:
mp4 = open('test_recording_1.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [4]:
!ffmpeg -i test_recording_1.mp4 -vf "select=gt(scene\,0.003),setpts=N/(25*TB)" -y output.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [5]:
mp4 = open('output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [6]:
!ffmpeg -i test_recording_1.mp4 -vcodec copy -acodec copy -f null /dev/null 2>&1 | grep -o -P '(frame=  ).*( fps)'|grep -o -E '[0-9]+' |tail -1 | sed -e 's/^0\+//'

284


In [7]:
!ffprobe -v 0 -of compact=p=0 -select_streams 0 -show_entries stream=r_frame_rate test_recording_1.mp4

r_frame_rate=90000/3001


In [8]:
!ffmpeg -i output.mp4 -vcodec copy -acodec copy -f null /dev/null 2>&1 | grep -o -P '(frame=  ).*( fps)'|grep -o -E '[0-9]+' |tail -1 | sed -e 's/^0\+//'

96


In [9]:
!ffprobe -v 0 -of compact=p=0 -select_streams 0 -show_entries stream=r_frame_rate output.mp4

r_frame_rate=90000/3001


In [10]:
90000/3001

29.990003332222592

In [11]:
def images_to_video(image_list, fps, video_name):

    movie_clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_list, fps)
    movie_clip.write_videofile(video_name)

In [12]:
def convert_mp4_to_avi(file_name, src_path, tgt_path):
    src_qualified_file_name = os.path.join(src_path, file_name)
    tgt_file = file_name.split(".mp4")[0] + ".avi"
    tgt_qualified_file_name = os.path.join(tgt_path, tgt_file)
    !rm -rf {tgt_qualified_file_name}
    ff = ffmpy.FFmpeg(inputs  = {src_qualified_file_name: None},
                      outputs = {tgt_qualified_file_name: None}
                     )
    ff.run()

In [13]:
vidcap = cv2.VideoCapture('output.mp4')
success,image = vidcap.read()
count = 1
video_counter = 1
image_list = []
while success:
  image_file_name = f"output_frame_{str(count).zfill(3)}.jpg"
  cv2.imwrite(image_file_name, image)     # save frame as JPEG file
  success,image = vidcap.read()
  #print('Read a new frame: ', success)
  image_list.append(image_file_name)
  if count % 8 == 0:
      video_name = f'output_video_seq_{str(video_counter).zfill(3)}.mp4'
      images_to_video(image_list, 30, video_name)
      convert_mp4_to_avi(file_name = video_name, src_path = os.getcwd(), tgt_path = os.getcwd())
      video_counter += 1
      image_list = []
  count += 1

Moviepy - Building video output_video_seq_001.mp4.
Moviepy - Writing video output_video_seq_001.mp4



Moviepy - Done !
Moviepy - video ready output_video_seq_001.mp4
Moviepy - Building video output_video_seq_002.mp4.
Moviepy - Writing video output_video_seq_002.mp4



Moviepy - Done !
Moviepy - video ready output_video_seq_002.mp4
Moviepy - Building video output_video_seq_003.mp4.
Moviepy - Writing video output_video_seq_003.mp4



Moviepy - Done !
Moviepy - video ready output_video_seq_003.mp4
Moviepy - Building video output_video_seq_004.mp4.
Moviepy - Writing video output_video_seq_004.mp4



Moviepy - Done !
Moviepy - video ready output_video_seq_004.mp4
Moviepy - Building video output_video_seq_005.mp4.
Moviepy - Writing video output_video_seq_005.mp4



...

Moviepy - Done !
Moviepy - video ready output_video_seq_008.mp4
Moviepy - Building video output_video_seq_009.mp4.
Moviepy - Writing video output_video_seq_009.mp4



Moviepy - Done !
Moviepy - video ready output_video_seq_009.mp4
Moviepy - Building video output_video_seq_010.mp4.
Moviepy - Writing video output_video_seq_010.mp4



Moviepy - Done !
Moviepy - video ready output_video_seq_010.mp4
Moviepy - Building video output_video_seq_011.mp4.
Moviepy - Writing video output_video_seq_011.mp4



Moviepy - Done !
Moviepy - video ready output_video_seq_011.mp4
Moviepy - Building video output_video_seq_012.mp4.
Moviepy - Writing video output_video_seq_012.mp4



Moviepy - Done !
Moviepy - video ready output_video_seq_012.mp4


In [14]:
!ls -ltr output_video_seq*.mp4

-rw-r--r-- 1 root root 30932 Dec 13 04:24 output_video_seq_001.mp4
-rw-r--r-- 1 root root 34315 Dec 13 04:24 output_video_seq_002.mp4
-rw-r--r-- 1 root root 32255 Dec 13 04:24 output_video_seq_003.mp4
-rw-r--r-- 1 root root 29023 Dec 13 04:24 output_video_seq_004.mp4
-rw-r--r-- 1 root root 34787 Dec 13 04:24 output_video_seq_005.mp4
-rw-r--r-- 1 root root 35090 Dec 13 04:24 output_video_seq_006.mp4
-rw-r--r-- 1 root root 31297 Dec 13 04:24 output_video_seq_007.mp4
-rw-r--r-- 1 root root 32142 Dec 13 04:24 output_video_seq_008.mp4
-rw-r--r-- 1 root root 31857 Dec 13 04:24 output_video_seq_009.mp4
-rw-r--r-- 1 root root 34033 Dec 13 04:24 output_video_seq_010.mp4
-rw-r--r-- 1 root root 33621 Dec 13 04:24 output_video_seq_011.mp4
-rw-r--r-- 1 root root 30232 Dec 13 04:24 output_video_seq_012.mp4


In [15]:
sorted(glob.glob("output_video_seq*.mp4"))

['output_video_seq_001.mp4',
 'output_video_seq_002.mp4',
 'output_video_seq_003.mp4',
 'output_video_seq_004.mp4',
 'output_video_seq_005.mp4',
 'output_video_seq_006.mp4',
 'output_video_seq_007.mp4',
 'output_video_seq_008.mp4',
 'output_video_seq_009.mp4',
 'output_video_seq_010.mp4',
 'output_video_seq_011.mp4',
 'output_video_seq_012.mp4']

In [ ]:
#hello
mp4 = open('output_video_seq_001.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#hello/teacher
mp4 = open('output_video_seq_002.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#teacher
mp4 = open('output_video_seq_003.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#teacher
mp4 = open('output_video_seq_004.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#teacher
mp4 = open('output_video_seq_005.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#thank you
mp4 = open('output_video_seq_006.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#thank you
mp4 = open('output_video_seq_007.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#love?
mp4 = open('output_video_seq_008.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#love
mp4 = open('output_video_seq_009.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#class
mp4 = open('output_video_seq_010.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#class
mp4 = open('output_video_seq_011.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#class
mp4 = open('output_video_seq_012.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [16]:
resolution = 224
num_frames = 8
batch_size = 8
data = os.path.join('asl', 'test')

In [17]:
tflite = os.path.join(os.getcwd(), 'model_1.tflite')
video_path = os.path.join(os.getcwd(), 'output_video_seq_001.avi')

In [18]:
# Load TFLite Model
# Create the interpreter and signature runner
interpreter = tf.lite.Interpreter(model_path=tflite)
runner = interpreter.get_signature_runner()
init_states = {
    name: tf.zeros(x['shape'], dtype=x['dtype'])
    for name, x in runner.get_input_details().items()
}
del init_states['image']
runner.get_input_details().items()

dict_items([('image', {'name': 'serving_default_image:0', 'index': 55, 'shape': array([  1,   1, 224, 224,   3], dtype=int32), 'shape_signature': array([  1,   1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}), ('state_block0_layer0_pool_buffer', {'name': 'serving_default_state_block0_layer0_pool_buffer:0', 'index': 24, 'shape': array([ 1,  1,  1,  1, 40], dtype=int32), 'shape_signature': array([ 1,  1,  1,  1, 40], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}), ('state_block0_layer0_pool_frame_count', {'name': 'serving_default_state_block0_layer0_pool_frame_count:0', 'index': 26, 'shape': array([1], dtype

In [19]:
image_size = (resolution, resolution)

In [21]:
!mkdir asl
!mkdir asl/test
!mkdir asl/test/hello asl/test/teacher asl/test/class asl/test/thankyou asl/test/love

In [22]:
#label_map = {'class': 0, 'hello': 1, 'love': 2, 'teacher': 3, 'thankyou': 4}
label_map = sorted(os.listdir('/content/asl/test'))
label_map

['class', 'hello', 'love', 'teacher', 'thankyou']

In [23]:
def get_top_k(probs, k=5, label_map=label_map):
    """Outputs the top k model labels and probabilities on the given video."""
    top_predictions = tf.argsort(probs, axis=-1, direction='DESCENDING')[:k]
    top_labels = tf.gather(label_map, top_predictions, axis=-1)
    top_labels = [label.decode('utf8') for label in top_labels.numpy()]
    top_probs = tf.gather(probs, top_predictions, axis=-1).numpy()
    return tuple(zip(top_labels, top_probs))

In [24]:
def read_video_seg_and_get_label(video_name, num_frames):
    cap = cv2.VideoCapture(video_name)
    frames_queue = deque(maxlen=num_frames)
    p_time = 0
    while True:
        success, img = cap.read()
        if not success:
            break

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        frames_queue.append(img_rgb)
        if len(frames_queue) == num_frames:

            img_to_tensor = tf.convert_to_tensor(frames_queue, dtype=tf.uint8)
            # print(type(img_to_tensor))
            img_resize = tf.image.resize(img_to_tensor, image_size)
            img_norm = tf.cast(img_resize, tf.float32) / 255.
            clips = tf.split(img_norm[tf.newaxis], img_norm.shape[0], axis=1)

            # To run on a video, pass in one frame at a time
            states = init_states
            for clip in clips:
                # Input shape: [1, 1, 172, 172, 3] ---> 224, 224 for a2 version
                outputs = runner(**states, image=clip)
                logits = outputs.pop('logits')[0]
                states = outputs

            probs = tf.nn.softmax(logits, axis=-1)
            #print(probs)
            top_prob = label_map[np.argmax(probs)]
            #print(top_prob)

        # FPS
        c_time = time.time()
        fps_ = 1/(c_time-p_time)
        p_time = c_time

        #print(fps_)

        #cv2_imshow(img)
    cap.release()
    cv2.destroyAllWindows()
    return top_prob

In [25]:
pred_list = []
for video in sorted(glob.glob("output_video_seq*.mp4")):
    video_path = os.path.join(os.getcwd(), video)
    pred_list.append(read_video_seg_and_get_label(video_path, num_frames))
print(pred_list)

['hello', 'hello', 'teacher', 'hello', 'love', 'teacher', 'teacher', 'teacher', 'class', 'love', 'class', 'class']


In [ ]:
pred_list = []
for video in sorted(glob.glob("output_video_seq*.mp4")):
    video_path = os.path.join(os.getcwd(), video)
    pred_list.append(read_video_seg_and_get_label(video_path, num_frames))
print(pred_list)

['hello', 'hello', 'teacher', 'hello', 'love', 'teacher', 'teacher', 'teacher', 'class', 'love', 'class', 'class']


In [26]:
!pip install nlpcloud

In [27]:
import nlpcloud

In [28]:
input_list = pred_list
clean_list = []
for idx in range(len(input_list) - 1):
    if input_list[idx] == input_list[idx + 1] and input_list[idx] not in clean_list:
        clean_list.append(input_list[idx])
    elif input_list[idx] != input_list[idx + 1] and input_list[idx] not in clean_list:
        clean_list.append(input_list[idx])
clean_list

['hello', 'teacher', 'love', 'class']

In [29]:
correction_string = " ".join(clean_list)
correction_string

'hello teacher love class'

In [30]:
client = nlpcloud.Client("finetuned-llama-2-70b", "d4274d3e168c5adbcc135f7125cbff8e76544dc3", True)
# Returns a json object.
#client.gs_correction("hello teacher thankyou love class")

client.gs_correction(correction_string)

{'correction': 'Hello teacher, I love this class.'}

In [34]:
eng_text = client.gs_correction(correction_string)['correction']
eng_text

'Hello teacher, I love this class!'

In [31]:
!pip install transformers sentencepiece -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.1 MB/s eta 0:00:00


In [32]:
# Importing the pipeline function from the transformers library
from transformers import pipeline

In [33]:
# Creating a Text2TextGenerationPipeline for language translation
pipe = pipeline(task='text2text-generation', model='facebook/m2m100_418M')

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [35]:
# Converting to French
pipe(eng_text, forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='fr'))

[{'generated_text': 'Bonjour professeur, j’aime cette classe !'}]

In [36]:
# Converting to Spanish
pipe(eng_text, forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='es'))

[{'generated_text': 'Hola profesora, me encanta esta clase!'}]

In [37]:
# Converting to Hindi
pipe(eng_text, forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='hi'))

[{'generated_text': 'हाय शिक्षक, मुझे यह कक्षा पसंद है!'}]